In [1]:
import numpy as np
import os
import sys
import glob

import h5py



# %qtconsole
%matplotlib qt5
%matplotlib auto
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
# import matplotlib
# matplotlib.use('TkAgg')


from multiprocessing import Process
import threading
# import getpass


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: Qt5Agg


In [2]:
%qtconsole

In [2]:
    
vid_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'

# LOAD IN DATA FROM PICKLE OR FEATHER (SKIP ALL NEXT DATAFRAME SECTIONS)
# start_time = time.time()
# df = pd.read_pickle(vid_locations + 'AllTracks')
# print('read in all trackways pickle -- duration: ', time.time()-start_time)
# start_time = time.time()
# longtracks = pd.read_pickle(vid_locations + 'LongTracks')
# print('read in all trackways pickle -- duration: ', time.time()-start_time)

## Build file list of tracked data

Important to note. For this first pass analysis lets only look at videos from camera 1.

In [4]:
file_list = []
# for root, dirs, files in os.walk(vid_locations):
#     for file in files:
#         if(os.path.splitext(file)[1] == '.avi' and # only avi files
#            os.path.basename(file)[0] != '5'):      # dont track video 5
#             file_list.append(os.path.join(root, file))
# len(file_list)


# # glob is like dir in matlab, it searches for files
# file_list = glob.glob(os.path.join(vid_locations, '**/*12-0000.avi'))
# file_list = glob.glob(os.path.join(vid_locations, '*clear/*12-0000.avi'))
# file_list = glob.glob(os.path.join(vid_locations, '2mm_clear/*12-0000.avi'))
file_list = glob.glob(os.path.join(vid_locations, '**/**/*0000.mp4'))
# file_list = glob.glob(os.path.join(vid_locations, 'Tunnel_20180313-14/**/*0000.avi'))
file_list = sorted(file_list)

print('Total Number of Videos: ',len(file_list))
# file_list = sorted(file_list)[0:4] #[::20] #[0:3000:5]
# file_list = file_list[0:15] +file_list[278:292] +file_list[578:592] +file_list[878:902]
                
len(file_list)

Total Number of Videos:  9594


9594

In [6]:
def load_predictions(h_path):
    pred_path = h_path.split('.')[0] + '_predictions.h5'
    joints = {}
    
    if os.path.exists(pred_path):
        hf = h5py.File(pred_path,'r')
        joint_loc = hf['positions_pred'][()].astype(np.float32) #hf.get('positions_pred')
        joint_conf = hf['conf_pred'][()] #hf.get('conf_pred')
        hf.close
    
#         # LOAD JOINT TRACKING DATA
#         # legs
#         for joint_num in range(0,6):
#             joints['tarsus%i_x'%joint_num]=joint_loc[:,0,27-4*joint_num]
#             joints['tarsus%i_y'%joint_num]=joint_loc[:,1,27-4*joint_num]
#             joints['tarsus%i_conf'%joint_num]=joint_conf[:,27-4*joint_num]
#         # antennae
#         for anten_num in range(0,2):
#             joints['antenna%i_x'%anten_num]=joint_loc[:,0,28+4*anten_num]
#             joints['antenna%i_y'%anten_num]=joint_loc[:,1,28+4*anten_num]
#             joints['antenna%i_conf'%anten_num]=joint_conf[:,28+4*anten_num]
#         # thorax & neck
#         joints['thorax_x']=joint_loc[:,0,2]
#         joints['thorax_y']=joint_loc[:,1,2]
#         joints['thorax_conf']=joint_conf[:,2]
#         joints['neck_x']=joint_loc[:,0,1]
#         joints['neck_y']=joint_loc[:,1,1]
#         joints['neck_conf']=joint_conf[:,1]
        
        # LOAD JOINT TRACKING DATA FOR ****10*** TRACKED POINTS
        # legs
        for joint_num in range(0,6):
            joints['joint%i_x'%joint_num]=joint_loc[:,0,7-joint_num]
            joints['joint%i_y'%joint_num]=joint_loc[:,1,7-joint_num]
            joints['joint%i_conf'%joint_num]=joint_conf[:,7-joint_num]
        # antennae
        for anten_num in range(0,2):
            joints['antenna%i_x'%anten_num]=joint_loc[:,0,8+anten_num]
            joints['antenna%i_y'%anten_num]=joint_loc[:,1,8+anten_num]
            joints['antenna%i_conf'%anten_num]=joint_conf[:,8+anten_num]
        # thorax & neck
        joints['thorax_x']=joint_loc[:,0,1]
        joints['thorax_y']=joint_loc[:,1,1]
        joints['thorax_conf']=joint_conf[:,1]
        joints['neck_x']=joint_loc[:,0,0]
        joints['neck_y']=joint_loc[:,1,0]
        joints['neck_conf']=joint_conf[:,0]
        
    return joints

def plot_predictions(joints, all_joints, fr_id):
    
    # for each key (joint name):
    for joint_name in all_joints:
        norm2 = colors.Normalize(vmin=0, vmax=1)
        plt.scatter(joints['%s_x'%joint_name][fr_id], joints['%s_y'%joint_name][fr_id], c = joints['%s_conf'%joint_name][fr_id], s = 20, cmap = cm.bwr,
                       edgecolor = 'none', norm=norm2)# '.g')
        
    return
    


In [ ]:
plt.close('all')


def key_event(e):
    global curr_pos
    global close_fig

    if e.key == "right":
        curr_pos = curr_pos + 1
    elif e.key == "left":
        curr_pos = curr_pos - 1
    elif e.key == "w":
        close_fig = True
        return
    else:
        return
    curr_pos = curr_pos % len(frames)

    frame_toshow = np.swapaxes(np.tile(frames[curr_pos].astype(np.uint8),(3,1,1)),0,2)
    ax.cla()
    plt.imshow(frame_toshow)
    plot_predictions(joints, all_joints, curr_pos)
    plt.axis('off')
    plt.text(10,10, 'Fr: %i'%curr_pos, color = 'w')
    plt.title('%s -- %s -- %s'%(folder_bits[-2], ('_').join(folder_bits[-1].split('_')[0:2]), h_path.split('/')[-1][:-3]), loc = 'left')
    fig.canvas.draw()

def close_out_of_program():
    continueon = input('Press any key to exit:  ')
    print('\nClosing down viewer...')
    return #close_up_shop
    
    
i=threading.Thread(target=close_out_of_program, args = ())
i.start()

    
print('*** PRESS "w" TO CLOSE A FIGURE AND MOVE TO THE NEXT ANT H5 ***')
for kk, file in enumerate(file_list[13:]):
    folder = file.split('.mp4')[0]
    folder_bits = folder.split('/')
    a_files = glob.glob(os.path.join(folder, 'ant*[0-9].h5'))
    print(folder)

    for ff, h_path in enumerate(sorted(a_files)):
        close_fig = False
        print(' --- %s'%h_path.split('/')[-1])
        hf = h5py.File(h_path,'r')
        frames= hf['box'] 
        hf.close
        
        # load joint predictions
        joints = load_predictions(h_path)
        # find all joint names
        all_keys = joints.keys()
        all_joints = [ key[:-2] for key in all_keys if '_x' in key]

        curr_pos = 0

        fig= plt.figure()
        mngr = plt.get_current_fig_manager()
        mngr.window.setGeometry(1550,300,1000,1045)
        ax=fig.add_subplot(111)
        plt.imshow(np.swapaxes(np.tile(frames[0].astype(np.uint8),(3,1,1)),0,2))
        plot_predictions(joints, all_joints, curr_pos)
        plt.text(10,10, 'Fr: %i'%curr_pos, color = 'w')
        plt.title('%s -- %s -- %s'%(folder_bits[-2], ('_').join(folder_bits[-1].split('_')[0:2]), h_path.split('/')[-1][:-3]), loc = 'left')
        plt.axis('off')
        fig.canvas.mpl_connect('key_press_event', key_event)
        plt.show(block=False)
        
        fig_open = plt.fignum_exists(fig.number)
        while (not close_fig) and fig_open:
            plt.pause(3)
            if not i.isAlive():
#                 print('User input has ended. Closing...')
                break
        plt.close(fig)

        if not i.isAlive():
#             print('User input has ended. Closing...')
            break
            
    if not i.isAlive():
#         print('User input has ended. Closing...')
        break
#     continueon = input('Hit "N" or "n" to exit:  ')
# #     continueon = getpass.getpass('continue to next file [Y or N]?')
#     cursor_up = '\x1b[1A'
#     erase_line = '\x1b[2K'
#     print(cursor_up + erase_line)
# #     screen_code = "\033[1A[\033[2K"
# #     sys.stdout.write( screen_code)
# #     print("\033[A                     \033[A")
#     if (continueon == 'N') or (continueon == 'n'):
#         break
        
print('All done!')
        

*** PRESS "w" TO CLOSE A FIGURE AND MOVE TO THE NEXT ANT H5 ***
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_083810_16276736-0000
 --- ant0.h5


In [ ]:
def set_up_figure():
    fig,ax = plt.subplots()
    return fig, ax

for n in range(0,2):
    fig, ax = set_up_figure()
    ax.plot(n*2,n*4,'.k')
    plt.show(block=True)